# **Homework 1: Introduction for MLOps Zoomcamp 2024**

In [99]:
import pandas as pd

In [100]:
# Read the .parquet file for the month of january.
january = pd.read_parquet("/content/yellow_tripdata_2023-01.parquet")
january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [101]:
# Get the number of columns
num_columnas = january.shape[1]

print("Number of columns:", num_columnas)

Number of columns: 19


### **Q1. Downloading the data**

Number of columns: 19

In [102]:
# Calculate the duration of each trip in minutes and the Standard deviation of trip in january.
def calculate_duration(data):
    data["duration"] = data["tpep_dropoff_datetime"] - data["tpep_pickup_datetime"]
    data.duration = data.duration.apply(lambda td: td.total_seconds() / 60)
    return data

january = calculate_duration(january)

print("Standard deviation of trip length in January:", round(january.duration.std(),2),"minutes")

Standard deviation of trip length in January: 42.59 minutes


### **Q2. Computing duration**

42.59 minutes

In [103]:
#Filter the DataFrame to include only records with trip durations between 1 and 60 minutes.

def duration_filter(data):
    data_filter = data.loc[(data["duration"]>=1) & (data["duration"] <= 60), :]
    return data_filter.reset_index(drop=True)


# Filter the January DataFrame
january_filter = duration_filter(january)

# Calculate the fraction of records remaining after eliminating outliers
fraction_left = round(january_filter.shape[0] / january.shape[0] * 100)

print("Fraction of records remaining after eliminating outliers:", fraction_left, "%")

Fraction of records remaining after eliminating outliers: 98 %


### **Q3. Dropping outliers**

The fraction is 98%

In [104]:
from sklearn.feature_extraction import DictVectorizer

In [105]:
#Encodes the categorical columns of a DataFrame using one-hot encoding.

def encode_categorical_columns(data, categorical_columns):

    # Convert categorical columns to string type
    data[categorical_columns] = data[categorical_columns].astype(str)

    # Convert DataFrame to a list of dictionaries
    train_dicts = data[categorical_columns].to_dict(orient='records')

    # Initialize and fit the DictVectorizer
    dv = DictVectorizer()
    X_train = dv.fit_transform(train_dicts)

    return X_train, dv

# Define categorical columns
categorical_columns = ["PULocationID", "DOLocationID"]
X_train, dv = encode_categorical_columns(january_filter, categorical_columns)

In [106]:
print("The dimensionality of this matrix (number of columns) are:",X_train.shape[1])

The dimensionality of this matrix (number of columns) are: 515


### **Q4. One-hot encoding**

The number of columns are 515

In [107]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [108]:
y_train = january_filter.duration.values

# Fit a linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict the target values
y_pred = lr.predict(X_train)

# Calculate the root mean squared error (RMSE)
rmse = mean_squared_error(y_train, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 7.649261929771859


### **Q5. Training a model**

RMSE: 7.64


In [109]:
# Read the .parquet file for the month of February
february = pd.read_parquet("/content/yellow_tripdata_2023-02.parquet")
february.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [110]:
# Calculates the duration of each trip in minutes
february["duration"] = february["tpep_dropoff_datetime"] - february["tpep_pickup_datetime"]
february.duration = february.duration.apply(lambda td: td.total_seconds() / 60)

# Filters trips with a duration between 1 and 60 minutes
february_filter = february.loc[(february["duration"]>=1) & (february["duration"] <= 60), :]

# Converts categorical columns to string type
categorical = ["PULocationID", "DOLocationID"]
february_filter[categorical] = february_filter[categorical].astype(str)

# Converts the filtered DataFrame to a list of dictionaries
val_dicts = february_filter[categorical].to_dict(orient="records")

# Transforms the validation data using the same DictVectorizer that was fitted with the training data.
X_valid = dv.transform(val_dicts)
X_valid.shape

<ipython-input-110-b7f3529eb3ca>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  february_filter[categorical] = february_filter[categorical].astype(str)


(2855951, 515)

In [111]:
# The values of the "duration" column of the filtered DataFrame february_filter are extracted.
y_valid = february_filter["duration"].values

# The transformed X_valid validation data are used to make predictions using the lr model.
y_pred_val = lr.predict(X_valid)

# Calculate RMSE
rmse_val = mean_squared_error(y_valid, y_pred_val, squared=False)
print("Root Mean Squared Error (RMSE):", rmse_val)

Root Mean Squared Error (RMSE): 7.811818933419717


### **Q6. Evaluating the model**

RMSE: 7.81
